In [6]:
import os
from dotenv import load_dotenv

from langchain_community.document_loaders import PyPDFLoader, PyMuPDFLoader
from langchain_community.document_loaders import DirectoryLoader
from langchain_community.document_loaders import TextLoader

from langchain_tavily import TavilySearch
from langchain_text_splitters import RecursiveCharacterTextSplitter
from tqdm import tqdm

/Users/siddhigupta/Desktop/Arvia-Ai/Arvia/.venv/lib/python3.14/site-packages/langchain_tavily/tavily_research.py:97: UserWarning: Field name "output_schema" in "TavilyResearch" shadows an attribute in parent "BaseTool"
  class TavilyResearch(BaseTool):  # type: ignore[override, override]
/Users/siddhigupta/Desktop/Arvia-Ai/Arvia/.venv/lib/python3.14/site-packages/langchain_tavily/tavily_research.py:97: UserWarning: Field name "stream" in "TavilyResearch" shadows an attribute in parent "BaseTool"
  class TavilyResearch(BaseTool):  # type: ignore[override, override]


In [2]:
all_pdf_documents = []
dir_loader=DirectoryLoader(
    "../data/remedies",
    glob="**/*.pdf", 
    loader_cls= PyMuPDFLoader, 
    use_multithreading=True,
    show_progress=True

)



pdf_documents=dir_loader.lazy_load()

for doc in dir_loader.lazy_load():   
    doc.metadata["source_type"] = "Authorotative"
    doc.metadata["priority"] = "1"
    all_pdf_documents.append(doc)

100%|██████████| 2/2 [00:01<00:00,  1.22it/s]


In [ ]:
def split_documents(pdf_documents,chunk_size=1000,chunk_overlap=200):

    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=chunk_size,
        chunk_overlap=chunk_overlap,
        length_function=len,
        separators=["\n\n", "\n", " ", ""]
    )
    split_docs = text_splitter.split_documents(pdf_documents)
    print(f"Split {len(pdf_documents)} documents into {len(split_docs)} chunks")
    
    # Show example of a chunk
    if split_docs:
        print(f"\nExample chunk:")
        print(f"Content: {split_docs[0].page_content[:200]}...")
        print(f"Metadata: {split_docs[0].metadata}")
    
    return split_docs


In [4]:
chunks=split_documents(all_pdf_documents)
chunks


Example chunk:
Content: i
STANDARD TREATMENT GUIDELINES
A Manual for Medical Therapeutics
First Edition, 2013
Gujarat Medical Services Corporation Limited
Health & Family Welfare Department
Government of Gujarat...
Metadata: {'producer': 'Nitro Pro 8  (8. 1. 1. 3)', 'creator': 'Nitro Pro', 'creationdate': '2013-11-16T11:06:14+05:30', 'source': '../data/remedies/standard-treatment-guidelines.pdf', 'file_path': '../data/remedies/standard-treatment-guidelines.pdf', 'total_pages': 431, 'format': 'PDF 1.7', 'title': 'STANDARD TREATMENT GUIDELINES', 'author': 'abc', 'subject': '', 'keywords': '', 'moddate': '2014-01-03T12:44:35+05:30', 'trapped': '', 'modDate': "D:20140103124435+05'30'", 'creationDate': "D:20131116110614+05'30'", 'page': 0, 'source_type': 'Authorotative', 'priority': '1'}


[Document(metadata={'producer': 'Nitro Pro 8  (8. 1. 1. 3)', 'creator': 'Nitro Pro', 'creationdate': '2013-11-16T11:06:14+05:30', 'source': '../data/remedies/standard-treatment-guidelines.pdf', 'file_path': '../data/remedies/standard-treatment-guidelines.pdf', 'total_pages': 431, 'format': 'PDF 1.7', 'title': 'STANDARD TREATMENT GUIDELINES', 'author': 'abc', 'subject': '', 'keywords': '', 'moddate': '2014-01-03T12:44:35+05:30', 'trapped': '', 'modDate': "D:20140103124435+05'30'", 'creationDate': "D:20131116110614+05'30'", 'page': 0, 'source_type': 'Authorotative', 'priority': '1'}, page_content='i\nSTANDARD TREATMENT GUIDELINES\nA Manual for Medical Therapeutics\nFirst Edition, 2013\nGujarat Medical Services Corporation Limited\nHealth & Family Welfare Department\nGovernment of Gujarat'),
 Document(metadata={'producer': 'Nitro Pro 8  (8. 1. 1. 3)', 'creator': 'Nitro Pro', 'creationdate': '2013-11-16T11:06:14+05:30', 'source': '../data/remedies/standard-treatment-guidelines.pdf', 'file_

In [6]:
from langchain_community.vectorstores import FAISS
from langchain_huggingface import HuggingFaceEmbeddings


embeddings = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")

#Load existing db
existing_db = FAISS.load_local(
    "../arvia_medical_index", 
    embeddings, 
    allow_dangerous_deserialization=True
)

# Temporary db
new_db = FAISS.from_documents(chunks, embeddings)

print(f"Chunks in Merck DB: {len(existing_db.docstore._dict)}")

#mergingg
existing_db.merge_from(new_db)

print(f"Total chunks after merge: {len(existing_db.docstore._dict)}")

# save combined
existing_db.save_local("arvia_medical_index_v2")

print("✅ New remedies merged into the existing medical index!")

Chunks in Merck DB: 19278
Total chunks after merge: 20678
✅ New remedies merged into the existing medical index!


In [2]:
from langchain_community.document_loaders.csv_loader import CSVLoader
from langchain_community.document_loaders import DirectoryLoader
all_csv_documents = []
dir_loader=DirectoryLoader(
    "../data/remedies",
    glob="**/*.csv", 
    loader_cls= CSVLoader, 
    use_multithreading=True,
    show_progress=True

)



csv_documents=dir_loader.lazy_load()

for doc in dir_loader.lazy_load():   
    doc.metadata["source_type"] = "Remedies"
    doc.metadata["priority"] = "1"
    all_csv_documents.append(doc)

100%|██████████| 3/3 [00:00<00:00, 12.54it/s]


In [4]:
def split_documents(pdf_documents,chunk_size=1000,chunk_overlap=200):

    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=chunk_size,
        chunk_overlap=chunk_overlap,
        length_function=len,
        separators=["\n\n", "\n", " ", ""]
    )
    split_docs = text_splitter.split_documents(pdf_documents)
    print(f"Split {len(pdf_documents)} documents into {len(split_docs)} chunks")
    
    # Show example of a chunk
    if split_docs:
        print(f"\nExample chunk:")
        print(f"Content: {split_docs[0].page_content[:200]}...")
        print(f"Metadata: {split_docs[0].metadata}")
    
    return split_docs

In [7]:
csv_chunks=split_documents(all_csv_documents)
csv_chunks

Split 16927 documents into 39977 chunks

Example chunk:
Content: Name of Item: Ginger(Zingiber officinale Rose)
Health Issue: Indigestion
Home Remedy: 5 gm. crushed rhizome with salt or jaggery twice daily before meal.
Yogasan: https://bostcdn.bostinnovation.com/wh...
Metadata: {'source': '../data/remedies/Home Remedies.csv', 'row': 0, 'source_type': 'Remedies', 'priority': '1'}


[Document(metadata={'source': '../data/remedies/Home Remedies.csv', 'row': 0, 'source_type': 'Remedies', 'priority': '1'}, page_content='Name of Item: Ginger(Zingiber officinale Rose)\nHealth Issue: Indigestion\nHome Remedy: 5 gm. crushed rhizome with salt or jaggery twice daily before meal.\nYogasan: https://bostcdn.bostinnovation.com/which_exercise_helps_gastric_problem.jpg'),
 Document(metadata={'source': '../data/remedies/Home Remedies.csv', 'row': 1, 'source_type': 'Remedies', 'priority': '1'}, page_content='Name of Item: ANAR ( Punica granntum Linn., Dadima)\nHealth Issue: Indigestion\nHome Remedy: 10 ml. juice of fruit, 1 gm. black\x02salt or fried Jira powder with honey or sugar to hold inside m ou th for so m etim e b efo re food.\nYogasan: https://bostcdn.bostinnovation.com/which_exercise_helps_gastric_problem.jpg'),
 Document(metadata={'source': '../data/remedies/Home Remedies.csv', 'row': 2, 'source_type': 'Remedies', 'priority': '1'}, page_content='Name of Item: DALCHINI (

In [ ]:
from langchain_community.vectorstores import FAISS
from langchain_huggingface import HuggingFaceEmbeddings


embeddings = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")

#Load existing db
existing_db = FAISS.load_local(
    "arvia_medical_index_v2", 
    embeddings, 
    allow_dangerous_deserialization=True
)

# Temporary db
new_db = FAISS.from_documents(csv_chunks, embeddings)

print(f"Chunks in Merck DB: {len(existing_db.docstore._dict)}")

#mergingg
existing_db.merge_from(new_db)

print(f"Total chunks after merge: {len(existing_db.docstore._dict)}")

# save combined
existing_db.save_local("arvia_medical_index_v3")

print("✅ New remedies merged into the existing medical index!")

Chunks in Merck DB: 20678
Total chunks after merge: 60655
✅ New remedies merged into the existing medical index!
